# Simulación de Arribos de Aviones a AEP

Este notebook contiene la resolución de los problemas planteados sobre simulación de arribos de aviones mediante métodos de Monte Carlo.  
Incluye experimentos con distintos valores de λ, análisis de congestión, desvíos y posibles políticas de control de tráfico aéreo.  

---
# Simulación de Arribos de Aviones a AEP

Este notebook contiene la resolución de los problemas planteados sobre simulación de arribos de aviones mediante métodos de Monte Carlo.  
Incluye experimentos con distintos valores de λ, análisis de congestión, desvíos y posibles políticas de control de tráfico aéreo.  

---

## 0. Setup de la simulación

En esta sección se definen:  
- Parámetros globales (velocidades, distancias, tiempos).  
- Funciones auxiliares (generar arribos, simular dinámica, métricas, visualización).  
- Una función principal `simular(...)` que permita correr la simulación variando λ y otras condiciones.  
De este modo, el resto de las secciones solo necesitan llamar a estas funciones.

In [ ]:
# ==========================
# Imports y parámetros globales
# ==========================
import numpy as np
import matplotlib.pyplot as plt
import random

# --------------------------
# Parámetros básicos
# --------------------------
DISTANCIA_INICIAL = 100    # millas náuticas (mn) desde donde aparece el avión
VEL_MAX = 250 / 60         # nudos → mn/min (ej: 250 nudos = 250/60 mn/min)
VEL_MIN = 180 / 60         # nudos → mn/min
TIEMPO_TOTAL = 24 * 60     # duración de la simulación en minutos (1 día)
DT = 1                     # paso de simulación en minutos

# Distancia mínima y gaps de seguridad
SEPARACION_SEGURA = 5      # mn
GAP_TIEMPO = 10            # minutos (mínimo entre arribos seguros)

# Condiciones especiales
PROB_INTERRUPCION = 0.1    # día ventoso: 10% de probabilidad
CIERRE_TORMENTA = 30       # cierre en minutos
MAX_ATRASO = 90            # minutos, luego se desvía a Montevideo


In [ ]:
# ==========================
# Funciones auxiliares
# ==========================

def generar_arribos(lam, tiempo_total=TIEMPO_TOTAL, seed=None):
    """
    Genera tiempos de arribo de aviones como proceso de Poisson.
    """
    if seed is not None:
        np.random.seed(seed)
    
    arribos = []
    for t in range(tiempo_total):
        if np.random.rand() < lam:
            arribos.append(t)
    return arribos

In [ ]:
def reinsertar_por_viento(arribo, ultimo_aterrizaje):
    """
    Reinserta un avión que tuvo que abortar aterrizaje por viento.
    Busca un hueco >= GAP_TIEMPO y a más de SEPARACION_SEGURA mn.
    
    Aquí simplificamos: el avión aborta y se reprograma su arribo
    como si fuera +GAP_TIEMPO minutos después del último aterrizaje.
    """
    nuevo_arribo = max(arribo, ultimo_aterrizaje + GAP_TIEMPO)
    return nuevo_arribo


In [ ]:
def simular_dinamica(arribos, condiciones=None):
    """
    Simula la dinámica de aviones desde arribo hasta aterrizaje o desvío.
    
    condiciones (dict):
        {"viento": True, "tormenta": (inicio, fin)}
    """
    resultados = {
        "aterrizados": 0,
        "desviados": 0,
        "atrasos": [],
        "congestion": 0,
        "trayectorias": []
    }
    
    ultimo_aterrizaje = -np.inf
    
    for arribo in arribos:
        atraso = 0
        desviado = False
        arribo_real = arribo
        
        # --------------------------
        # Tormenta (cierre de AEP)
        # --------------------------
        if condiciones and condiciones.get("tormenta"):
            t_ini, t_fin = condiciones["tormenta"]
            if t_ini <= arribo <= t_fin:
                # el avión debe esperar hasta reapertura
                atraso += (t_fin - arribo)
                arribo_real = t_fin
        
        # --------------------------
        # Día ventoso (interrupción)
        # --------------------------
        if condiciones and condiciones.get("viento"):
            if random.random() < PROB_INTERRUPCION:
                arribo_real = reinsertar_por_viento(arribo_real, ultimo_aterrizaje)
                atraso = arribo_real - arribo
        
        # --------------------------
        # Chequear separación mínima
        # --------------------------
        if arribo_real < ultimo_aterrizaje + GAP_TIEMPO:
            # congestión: esperar hasta que haya espacio
            atraso = (ultimo_aterrizaje + GAP_TIEMPO) - arribo
            arribo_real = ultimo_aterrizaje + GAP_TIEMPO
            resultados["congestion"] += 1
        
        # --------------------------
        # Desvío si atraso demasiado grande
        # --------------------------
        if atraso > MAX_ATRASO:
            resultados["desviados"] += 1
            desviado = True
        else:
            resultados["aterrizados"] += 1
            ultimo_aterrizaje = arribo_real
        
        # --------------------------
        # Guardar datos del avión
        # --------------------------
        resultados["atrasos"].append(atraso)
        resultados["trayectorias"].append({
            "arribo": arribo,
            "arribo_real": arribo_real,
            "atraso": atraso,
            "desviado": desviado
        })
    
    return resultados


In [ ]:
def calcular_metricas(resultados):
    """
    Calcula métricas clave de performance.
    """
    atrasos = np.array(resultados["atrasos"])
    return {
        "promedio_atraso": np.mean(atrasos) if len(atrasos) else 0,
        "desviados": resultados["desviados"],
        "prob_desvio": resultados["desviados"] / (len(atrasos) if len(atrasos) else 1),
        "frecuencia_congestion": resultados["congestion"] / (len(atrasos) if len(atrasos) else 1)
    }


In [ ]:
def visualizar(resultados, n=30):
    """
    Visualiza arribos vs aterrizajes reales (o desvíos).
    """
    tray = resultados["trayectorias"][:n]
    arribos = [t["arribo"] for t in tray]
    aterrizajes = [t["arribo_real"] for t in tray]
    colores = ["red" if t["desviado"] else "green" for t in tray]
    
    plt.figure(figsize=(10, 5))
    plt.scatter(arribos, range(len(arribos)), label="Arribos", marker="o", c="blue")
    plt.scatter(aterrizajes, range(len(aterrizajes)), label="Aterrizajes/Desvíos", marker="x", c=colores)
    plt.xlabel("Tiempo (min)")
    plt.ylabel("Aviones (orden de llegada)")
    plt.title("Simulación de arribos y aterrizajes con condiciones especiales")
    plt.legend()
    plt.show()


In [ ]:
# ==========================
# Función principal
# ==========================
def simular(lam, tiempo_total=TIEMPO_TOTAL, condiciones=None, seed=None):
    """
    Función de alto nivel para correr simulaciones.
    """
    arribos = generar_arribos(lam, tiempo_total, seed)
    resultados = simular_dinamica(arribos, condiciones)
    metricas = calcular_metricas(resultados)
    return {
        "arribos": arribos,
        "resultados": resultados,
        "metricas": metricas
    }



## 1. Simulación de Monte Carlo básica de la fila de aviones

Implementar una simulación que modele la posición de cada avión en el tiempo y en el espacio, junto con un mecanismo de visualización.

In [ ]:
# Código: simulación base de arribos y posiciones de los aviones


In [ ]:
# Código: visualización de la cola de aviones en el tiempo/espacio


## 2. Cálculo de λ para 1 avión por hora

Si el promedio de arribos es de 1 avión por hora, determinar el valor de λ.


In [ ]:
# Código: cálculo de λ


## 3. Probabilidad de llegada de 5 aviones en una hora

- Usar simulación de Monte Carlo para estimar la probabilidad.  
- Verificar el resultado de manera analítica con la distribución de Poisson.


In [ ]:
# Código: simulación para estimar probabilidad


In [ ]:
# Código: verificación analítica usando Poisson


## 4. Congestión según distintos λ

Simular el sistema con λ ∈ {0.02, 0.1, 0.2, 0.5, 1}.  
Analizar:
- Frecuencia de congestión.  
- Atraso promedio de los aviones.  
- Frecuencia de desvíos a Montevideo.  
- Acompañar cada resultado con estimación de error.  
- Mostrar visualizaciones gráficas.


In [ ]:
# Código: simulación para distintos λ


In [ ]:
# Código: métricas de congestión y atrasos


In [ ]:
# Código: visualización comparativa


## 5. Escenario con día ventoso

Cada avión tiene 1/10 de probabilidad de tener que interrumpir su aterrizaje.  
Se debe simular:
- Estadísticas de atrasos en función de λ.  
- Desvíos a Montevideo.  
- Visualización de aviones (incluyendo los que se desvían).  
- Estimación de errores.


In [ ]:
# Código: simulación con interrupciones (día ventoso)


In [ ]:
# Código: métricas de performance bajo viento


In [ ]:
# Código: visualización de interrupciones y desvíos


## 6. Escenario con tormenta

Si AEP cierra sorpresivamente por 30 minutos:  
- Calcular consecuencias en desvíos, atrasos y otras métricas en función de λ.  
- Comparar con escenarios normales.


In [ ]:
# Código: simulación con cierre sorpresivo


In [ ]:
# Código: métricas y comparación con caso base


## 7. Políticas de vuelo alternativas

Explorar políticas simples basadas en información contemporánea (posiciones y velocidades actuales) para:  
- Reducir atrasos.  
- Disminuir o eliminar desvíos a Montevideo.  
- Analizar trade-off entre atraso promedio y frecuencia de desvíos.


In [ ]:
# Código: implementación de políticas alternativas


In [ ]:
# Código: evaluación de performance bajo políticas alternativas


## 8. Conclusiones

- Resumen de los principales hallazgos.  
- Comparación entre distintos escenarios de λ, viento y tormenta.  
- Discusión sobre posibles mejoras de políticas de control aéreo.


In [ ]:
# (Opcional) Gráficos finales comparativos / resumen visual
